In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import L_Layer_NN as nn

             Pclass                                               Name  \
PassengerId                                                              
1                 3                            Braund, Mr. Owen Harris   
2                 1  Cumings, Mrs. John Bradley (Florence Briggs Th...   
3                 3                             Heikkinen, Miss. Laina   
4                 1       Futrelle, Mrs. Jacques Heath (Lily May Peel)   
5                 3                           Allen, Mr. William Henry   

                Sex   Age  SibSp  Parch            Ticket     Fare Cabin  \
PassengerId                                                                
1              male  22.0      1      0         A/5 21171   7.2500   NaN   
2            female  38.0      1      0          PC 17599  71.2833   C85   
3            female  26.0      0      0  STON/O2. 3101282   7.9250   NaN   
4            female  35.0      1      0            113803  53.1000  C123   
5              male  35.0

TypeError: 'dict' object is not callable

In [2]:
data= 'titanic_train.csv'
test_data="titanic_test.csv"

In [3]:
X= pd.read_csv(data, index_col='PassengerId')
X_test=pd.read_csv(test_data, index_col="PassengerId")
Y= X.Survived
X=X.drop(['Survived'], axis=1)
print(X.head())

             Pclass                                               Name  \
PassengerId                                                              
1                 3                            Braund, Mr. Owen Harris   
2                 1  Cumings, Mrs. John Bradley (Florence Briggs Th...   
3                 3                             Heikkinen, Miss. Laina   
4                 1       Futrelle, Mrs. Jacques Heath (Lily May Peel)   
5                 3                           Allen, Mr. William Henry   

                Sex   Age  SibSp  Parch            Ticket     Fare Cabin  \
PassengerId                                                                
1              male  22.0      1      0         A/5 21171   7.2500   NaN   
2            female  38.0      1      0          PC 17599  71.2833   C85   
3            female  26.0      0      0  STON/O2. 3101282   7.9250   NaN   
4            female  35.0      1      0            113803  53.1000  C123   
5              male  35.0

In [4]:
# Get names of columns with missing values
cols_with_missing = [col for col in X.columns if X[col].isnull().any()]
print("cols with missing: ",cols_with_missing)
object_cols = [col for col in X.columns if X[col].dtype == "object"]
print("object cols: ",object_cols)
object_nunique = list(map(lambda col: X[col].nunique(), object_cols))
d = dict(zip(object_cols, object_nunique))
print("no. of unique entries: ", d)
low_cardinality_cols = [col for col in object_cols if X[col].nunique() < 10]
high_cardinality_cols = list(set(object_cols)-set(low_cardinality_cols))
print("low_cardinality_cols: ",low_cardinality_cols)
print("high_cardinality_cols: ", high_cardinality_cols)
numerical_cols= list(set(X.columns)-set(object_cols))
print("numerical_cols", numerical_cols)

cols with missing:  ['Age', 'Cabin', 'Embarked']
object cols:  ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']
no. of unique entries:  {'Name': 891, 'Sex': 2, 'Ticket': 681, 'Cabin': 147, 'Embarked': 3}
low_cardinality_cols:  ['Sex', 'Embarked']
high_cardinality_cols:  ['Ticket', 'Cabin', 'Name']
numerical_cols ['SibSp', 'Parch', 'Age', 'Pclass', 'Fare']


In [5]:
X= X.drop(high_cardinality_cols, axis=1)
X_test= X_test.drop(high_cardinality_cols, axis=1)

In [6]:
numerical_transformer = SimpleImputer(strategy='mean')
categorical_transformer = Pipeline(steps=[ ('imputer', SimpleImputer(strategy='most_frequent')),
                                           ('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(transformers=[('num', numerical_transformer, numerical_cols),
                                                 ('cat', categorical_transformer, low_cardinality_cols)])

In [7]:
X_transformed= preprocessor.fit_transform(X)
X_transformed= pd.DataFrame(StandardScaler().fit_transform(X_transformed))
X_transformed.index= X.index
print(X)
print(X_transformed)

             Pclass     Sex   Age  SibSp  Parch     Fare Embarked
PassengerId                                                      
1                 3    male  22.0      1      0   7.2500        S
2                 1  female  38.0      1      0  71.2833        C
3                 3  female  26.0      0      0   7.9250        S
4                 1  female  35.0      1      0  53.1000        S
5                 3    male  35.0      0      0   8.0500        S
...             ...     ...   ...    ...    ...      ...      ...
887               2    male  27.0      0      0  13.0000        S
888               1  female  19.0      0      0  30.0000        S
889               3  female   NaN      1      2  23.4500        S
890               1    male  26.0      0      0  30.0000        C
891               3    male  32.0      0      0   7.7500        Q

[891 rows x 7 columns]
                    0         1             2         3         4         5  \
PassengerId                            

In [8]:
X_test_trans= preprocessor.fit_transform(X_test)
X_test_trans= pd.DataFrame(StandardScaler().fit_transform(X_test_trans))
X_test_trans.index= X_test.index
print(X_test)
print(X_test_trans)

             Pclass     Sex   Age  SibSp  Parch      Fare Embarked
PassengerId                                                       
892               3    male  34.5      0      0    7.8292        Q
893               3  female  47.0      1      0    7.0000        S
894               2    male  62.0      0      0    9.6875        Q
895               3    male  27.0      0      0    8.6625        S
896               3  female  22.0      1      1   12.2875        S
...             ...     ...   ...    ...    ...       ...      ...
1305              3    male   NaN      0      0    8.0500        S
1306              1  female  39.0      0      0  108.9000        C
1307              3    male  38.5      0      0    7.2500        S
1308              3    male   NaN      0      0    8.0500        S
1309              3    male   NaN      1      1   22.3583        C

[418 rows x 7 columns]
                    0         1             2         3         4         5  \
PassengerId               

In [9]:
X_train, X_val, Y_train, Y_val= train_test_split(X_transformed, Y, test_size=0.3, random_state= 1)
X_train=X_train.T
X_val= X_val.T
train_indices=Y_train.index
val_indices=Y_val.index
Y_train= np.array([Y_train])
Y_val= np.array([Y_val])
X_test= X_test.T

In [10]:
### CONSTANTS ###
layers_dims = [10, 6, 4, 1] 

In [13]:
def L_layer_model(X, Y, layers_dims, learning_rate = 0.0075, num_iterations = 3000, print_cost=False):
    np.random.seed(1)
    costs = []                      
    parameters = nn.init_params(layers_dims)

    for i in range(0, num_iterations):
        AL, caches = nn.L_model_forward(X, parameters)
        cost = nn.compute_cost(AL, Y)
        grads = nn.L_model_backward(AL, Y, caches)
        parameters = nn.update_parameters(parameters, grads, learning_rate)
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
        if print_cost and i % 100 == 0:
            costs.append(cost)
    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per hundreds)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()
    return parameters

In [14]:
parameters = L_layer_model(X_train, Y_train, layers_dims, num_iterations = 2500, print_cost = True)

AssertionError: 

In [127]:
pred_train = nn.predict(X_train, Y_train, parameters)

AttributeError: module 'L_Layer_NN' has no attribute 'predict'

In [128]:
pred_val = nn.predict(X_val, Y_val, parameters)

AttributeError: module 'L_Layer_NN' has no attribute 'predict'